<a href="https://colab.research.google.com/github/mzwing/AI-related/blob/master/notebooks/NSFW_13B_sft_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prepare
!rm -rf sample_data
!mkdir -p NSFW_13B_sft-GGUF

In [ ]:
# get original model
!sudo apt-get install aria2 -y
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/zxbsmk/NSFW_13B_sft --depth 1
!cd NSFW_13B_sft && rm -rf pytorch_model-00001-of-00003.bin pytorch_model-00002-of-00003.bin pytorch_model-00003-of-00003.bin tokenizer.model .git

!echo -e "https://huggingface.co/zxbsmk/NSFW_13B_sft/resolve/main/pytorch_model-00001-of-00003.bin?download=true\n out=pytorch_model-00001-of-00003.bin\nhttps://huggingface.co/zxbsmk/NSFW_13B_sft/resolve/main/pytorch_model-00002-of-00003.bin?download=true\n out=pytorch_model-00002-of-00003.bin\nhttps://huggingface.co/zxbsmk/NSFW_13B_sft/resolve/main/pytorch_model-00003-of-00003.bin?download=true\n out=pytorch_model-00003-of-00003.bin\nhttps://huggingface.co/zxbsmk/NSFW_13B_sft/resolve/main/tokenizer.model?download=true\n out=tokenizer.model" > download.txt
!aria2c -c -x16 -d NSFW_13B_sft --input-file=download.txt

!rm -rf download.txt

In [ ]:
# prepare for llama.cpp quantise
!pip3 install sentencepiece gguf
!mkdir -p NSFW_13B_sft

!git clone https://github.com/ggerganov/llama.cpp --depth 1

In [ ]:
# convert to fp32
!cd llama.cpp && python3 ./convert-hf-to-gguf.py --outtype f32 --outfile ../NSFW_13B_sft-GGUF/NSFW_13B_sft.F32.gguf ../NSFW_13B_sft/

In [ ]:
# convert to fp16
!cd llama.cpp && python3 ./convert-hf-to-gguf.py --outtype f16 --outfile ../NSFW_13B_sft-GGUF/NSFW_13B_sft.F16.gguf ../NSFW_13B_sft/

In [ ]:
# uninstall to save space
!pip3 uninstall sentencepiece gguf -y
!pip cache purge
!rm -rf NSFW_13B_sft llama.cpp

In [ ]:
# prepare for CPU
!wget -O- https://apt.repos.intel.com/intel-gpg-keys/GPG-PUB-KEY-INTEL-SW-PRODUCTS.PUB | gpg --dearmor | sudo tee /usr/share/keyrings/oneapi-archive-keyring.gpg > /dev/null
!echo "deb [signed-by=/usr/share/keyrings/oneapi-archive-keyring.gpg] https://apt.repos.intel.com/oneapi all main" | sudo tee /etc/apt/sources.list.d/oneAPI.list
!sudo apt-get update
!sudo apt-get upgrade -y
!sudo apt-get install intel-oneapi-mkl -y

In [ ]:
# get compile result (CPU)
!aria2c -c -x16 https://github.com/MZWNET/actions/releases/download/llama_cpp-b2352/llama-b2352-bin-linux-avx2-intel-mkl-x64.zip
!mkdir -p llama.cpp-bin
!unzip llama-b2352-bin-linux-avx2-intel-mkl-x64.zip -d llama.cpp-bin
!mv -f llama.cpp-bin/main .
!mv -f llama.cpp-bin/quantize .
!rm -rf llama.cpp-bin llama-b2352-bin-linux-avx2-intel-mkl-x64.zip

In [ ]:
# prepare for GPU
!sudo apt-get install nvidia-cuda-toolkit -y

In [ ]:
# get compile result (GPU)
!sudo apt install aria2 -y
!aria2c -c -x16 https://github.com/MZWNET/actions/releases/download/llama_cpp-b2352/llama-b2352-bin-linux-avx2-cublas-cu121-x64.zip
!mkdir -p llama.cpp-bin
!unzip llama-b2352-bin-linux-avx2-cublas-cu121-x64.zip -d llama.cpp-bin
!mv -f llama.cpp-bin/main .
!mv -f llama.cpp-bin/quantize .
!rm -rf llama.cpp-bin llama-b2352-bin-linux-avx2-cublas-cu121-x64.zip

In [ ]:
# quantize
import concurrent.futures
import os
from tqdm import tqdm

parameters = [ "Q8_0", "Q6_K", "Q5_K_M", "Q5_K_S", "Q5_0", "Q4_K_M", "Q4_K_S", "Q4_0", "Q3_K_L", "Q3_K_M", "Q3_K_S", "Q2_K" ]

def run_command(param):
    os.system(f"cd NSFW_13B_sft-GGUF/ && ../quantize NSFW_13B_sft.F16.gguf NSFW_13B_sft.{param}.gguf {param}")

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(run_command, parameters), total=len(parameters)))

In [ ]:
# http upload
import concurrent.futures
from huggingface_hub import HfApi, create_repo

# create repo
create_repo("mzwing/NSFW_13B_sft-GGUF")

api = HfApi()

parameters = [ "F32", "F16", "Q8_0", "Q6_K", "Q5_K_M", "Q5_K_S", "Q5_0", "Q4_K_M", "Q4_K_S", "Q4_0", "Q3_K_L", "Q3_K_M", "Q3_K_S", "Q2_K" ]

def upload(params):
    api.upload_file(
        path_or_fileobj=f"NSFW_13B_sft-GGUF/NSFW_13B_sft.{params}.gguf",
        path_in_repo=f"NSFW_13B_sft.{params}.gguf",
        repo_id="mzwing/NSFW_13B_sft-GGUF",
    )

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    # Directly consume the iterator to ensure all futures are executed
    list(executor.map(upload, parameters))

In [ ]:
# test
!./main -ngl 33 -m NSFW_13B_sft-GGUF/NSFW_13B_sft.Q2_K.gguf --color --temp 0.7 --repeat_penalty 1.1 -n -1 -p "System: A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.\nHuman: {prompt}\nAssistant:Hi! What can I help you?\nHuman: "
!rm -rf main.log

In [ ]:
# git merge history
!git config --global credential.helper store
!huggingface-cli login
!git config --global user.email 'mzwing@mzwing.eu.org'
!git config --global user.name 'mzwing'
!rm -rf NSFW_13B_sft-GGUF/
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/mzwing/NSFW_13B_sft-GGUF
!cd NSFW_13B_sft-GGUF && git lfs install
!huggingface-cli lfs-enable-largefiles NSFW_13B_sft-GGUF/
!cd NSFW_13B_sft-GGUF/ && git branch backup-main
!cd NSFW_13B_sft-GGUF/ && git checkout --orphan new-main
!cd NSFW_13B_sft-GGUF/ && git add -A
!cd NSFW_13B_sft-GGUF/ && git commit -m "GGUF model commit (made with llama.cpp commit 26d6076)"
!cd NSFW_13B_sft-GGUF/ && git branch -D main
!cd NSFW_13B_sft-GGUF/ && git branch -m main
!cd NSFW_13B_sft-GGUF/ && git push -f origin main